# Mass balance of all glaciers: Timeseries

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats
import xarray as xr
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
period_start = "1975_1976"
period_stop = "2020_2021"
assert all("_" in period and len(period) == 9 for period in (period_start, period_stop))

## Define request

In [ ]:
y0_start, y1_start = map(int, period_start.split("_"))
y0_stop, y1_stop = map(int, period_stop.split("_"))
collection_id = "derived-gridded-glacier-mass-change"
request = {
    "variable": "glacier_mass_change",
    "product_version": "wgms_fog_2022_09",
    "format": "zip",
    "hydrological_year": [
        f"{y0}_{str(y1)[-2:]}"
        for y0, y1 in zip(range(y0_start, y0_stop + 1), range(y1_start, y1_stop + 1))
    ],
}

## Function to cache

In [ ]:
def spatial_sum(ds):
    return ds.sum(("latitude", "longitude"), keep_attrs=True)

## Download data

In [ ]:
ds = download.download_and_transform(
    collection_id,
    request,
    chunks={"hydrological_year": 1},
    transform_func=spatial_sum,
).compute()

# Customize attributes
ds["time"] = ds["time"].dt.year
ds["time"].attrs |= {"long_name": "Time", "units": "yr"}
for da in ds.data_vars.values():
    da.attrs["units"] += " $yr^{-1}$"
    da.attrs["long_name"] = da.attrs["long_name"].replace("_", " ").title()

# Compute and add cumulative
cumulative = ds["Glacier"].cumsum("time")
cumulative.attrs = {
    "units": ds["Glacier"].attrs["units"].split()[0],
    "long_name": f"Cumulative {ds['Glacier'].attrs['long_name']}",
}
ds["Cumulative"] = cumulative

## Plot timeseries

In [ ]:
for vars, title in zip(
    (["Glacier", "Uncertainty"], ["Glacier", "Cumulative"]),
    (
        "Total annual glacier mass changes and its uncertainties.",
        "The (cumulative) annual glacier mass changes at the global scale",
    ),
):
    fig, axs = plt.subplots(2, 1, layout="constrained")
    for ax, da in zip(axs, ds[vars].values()):
        da.plot(ax=ax)
        ax.grid()
        ax.set_title(da.attrs["long_name"])
    fig.suptitle(title)

## Plot timeseries on top of errors

In [ ]:
fig, ax = plt.subplots()
line = ds["Glacier"]
error = ds["Uncertainty"] / 2
ax.fill_between(line["time"], line - error, line + error, alpha=0.5)
line.plot(ax=ax, color="k")
ax.grid()
_ = ax.set_title("Glacier mass change and its uncertainty")
_ = fig.suptitle("Total annual glacier mass changes and its uncertainty.")

## Compute and plot trends

In [ ]:
da = ds["Cumulative"]

# Significance
_, p_value = scipy.stats.kendalltau(da["time"], da)
significance_level = 0.05
is_significant = p_value < significance_level
print(
    " ".join(
        [
            "The obtained linear trend of the mass changes of the glaciers over time",
            "is significant" if is_significant else "is not significant",
            f"at an alpha level of {significance_level}, i.e. a monotonic trend",
            "is present" if is_significant else "is not present",
            ".",
        ]
    )
)

# Plot
fig, ax = plt.subplots()
for label, degree in zip(
    (
        "Linear",
        "Quadratic",
    ),
    (1, 2),
):
    coeffs = da.polyfit("time", degree)["polyfit_coefficients"]
    equation = []
    for deg, coeff in coeffs.groupby("degree"):
        equation.append(
            f"{float(coeff):+.3f}{'x' if deg else ''}{f'$^{deg}$' if deg>1 else ''}"
        )
    label = f"{label}: {' '.join(equation[::-1])}"
    if degree == 1:
        label += f"\np-value = {p_value:.3f}"
    fit = xr.polyval(da["time"], coeffs)
    fit.plot(label=label, ax=ax)
da.plot(label="Cumulative", ax=ax)
ax.set_title("Cumulative glacier mass change")
ax.grid()
ax.legend(bbox_to_anchor=(1, 1), loc="upper left")
_ = fig.suptitle(
    "The (cumulative) annual glacier mass changes at the global scale and its linear/quadratic trend."
)